In [33]:
import numpy as np
import graph
import digraph
reload(graph)
reload(digraph)
from graph import Graph
from digraph import DiGraph
from collections import defaultdict
from itertools import tee, izip
import copy
import random
from random import shuffle
import time

## Preparation Functions/Classes

In [2]:
def read_instance_file(filename):
    '''
    ::param: filename: string, filename that indicates the location of instance data file
    ::return value: (specification, data)
    :: specification: dict, specification of the instance
    :: data: the numpy array with a list of edges and their cost, demand
    :: data: [vertex1 vertex2 cost demand]
    '''
    fd = open(inputfile)
    content = fd.readlines()
    content = [x.strip() for x in content] 
    specification = dict()
    for i in range(8):
        line = content[i].split(':')
        specification[line[0].strip()] = line[1].strip()
    # print(specification)
    data = list()
    for line in content[9:-1]:
        tmp = line.split()
        data.append([int(x.strip()) for x in tmp])
    data = np.array(data)
    fd.close()
    return specification, data

In [4]:
'''
filelist CARP_samples
egl-e1-A.dat  gdb10.dat  val1A.dat  val7A.dat
egl-s1-A.dat  gdb1.dat   val4A.dat
'''
inputfile = '../CARP_samples/egl-s1-A.dat'

In [5]:
# reload graph.py
reload(graph)

<module 'graph' from 'graph.pyc'>

## Read the graph into Data Structure

In [6]:
spec, data = read_instance_file(inputfile)
gf = Graph()
gf.load_from_data(data.tolist())

In [7]:
# print(gf)
print(spec)
# print(gf.get_tasks())

{'NON-REQUIRED EDGES': '115', 'VEHICLES': '7', 'CAPACITY': '210', 'NAME': 'egl-s1-A', 'TOTAL COST OF REQUIRED EDGES': '1394', 'REQUIRED EDGES': '75', 'VERTICES': '140', 'DEPOT': '1'}


In [8]:
capacity = int(spec['CAPACITY'])
depot = int(spec['DEPOT'])

In [9]:
# test
print(gf.get_shortest_path(1,18))

([1, 116, 117, 119, 120, 121, 122, 87, 86, 85, 84, 82, 80, 79, 78, 77, 46, 43, 44, 45, 34, 139, 33, 11, 12, 13, 17, 18], 423)


In [10]:
gf[10]

{9: {'cost': 12}, 15: {'cost': 13}, 19: {'cost': 28}}

## Initialization
### Path-Scanning Simple

In [11]:
def which_better(u1, u2, graph, load, capacity, depot):
    import random
    r_cq1 = graph[u1[0]][u1[1]]['cost']/graph[u1[0]][u1[1]]['demand']
    r_cq2 = graph[u2[0]][u2[1]]['cost']/graph[u2[0]][u2[1]]['demand']
    return_cost1 = graph.get_shortest_path(u1[1], depot)[1]
    return_cost2 = graph.get_shortest_path(u2[1], depot)[1]
    # print(u1,u2)
    '''
    if load < capacity/2:
        if r_cq1 > r_cq2:
            return u1
        elif r_cq1 < r_cq2:
            return u2
        elif return_cost1 > return_cost2:
            return u1
        elif return_cost1 < return_cost2:
            return u2
        else:     
            return random.choice([u1, u2])
    else:
        if r_cq1 < r_cq2:
            return u1
        elif r_cq1 > r_cq2:
            return u2
        elif return_cost1 < return_cost2:
            return u1
        elif return_cost1 > return_cost2:
            return u2
        else:
            return random.choice([u1, u2])
    '''
    # random better
    rule = random.choice([0,1,2,3,4])
    if rule == 0:
        if r_cq1 > r_cq2:
            return u1load
        elif r_cq1 < r_cq2:
            return u2
        else:     
            return random.choice([u1, u2])
    elif rule == 1:
        if return_cost1 > return_cost2:
            return u1
        elif return_cost1 < return_cost2:
            return u2
        else:     
            return random.choice([u1, u2])
    elif rule == 2:
        if r_cq1 < r_cq2:
            return u1
        elif r_cq1 > r_cq2:
            return u2
        else:     
            return random.choice([u1, u2])
    elif rule == 3:
        if return_cost1 < return_cost2:
            return u1
        elif return_cost1 > return_cost2:
            return u2
        else:
            return random.choice([u1, u2])
    elif rule == 4:
        if load < capacity/2:
            if return_cost1 > return_cost2:
                return u1
            elif return_cost1 < return_cost2:
                return u2
            else:     
                return random.choice([u1, u2])
        else:
            if return_cost1 < return_cost2:
                return u1
            elif return_cost1 > return_cost2:
                return u2
            else:
                return random.choice([u1, u2])

def ps_to_chromesome(solution):
    '''
    input pathscanning solution
    output chromesome rep
    '''
    result = list()
    for k,v in solution.iteritems():
        result += v
    return result
        
def path_scanning(graph, depot):
    k = 0
    R = defaultdict(dict)
    load = defaultdict(dict)
    cost = defaultdict(dict)
    free_task = set(graph.get_tasks())
    while len(free_task) > 0:
        k += 1
        R[k] = list()
        load[k], cost[k] = 0, 0
        end = depot
        u = None
        #print("############ beign #############")
        while True:
            if len(free_task) == 0:
                break
            d_min = float('inf')
            for f_task in free_task:
#                 if f_task == (106,105):
#                     print(load[k], graph[f_task[0]][f_task[1]]['demand'])
                if graph[f_task[0]][f_task[1]]['demand'] + load[k] > capacity:
                    continue
                if u == None:
                    u = f_task
                    d_min = graph.get_shortest_path(end, f_task[0])[1]
                d_tmp = graph.get_shortest_path(end, f_task[0])[1]
                # print(d_tmp,d_min, end,f_task, u)
                if d_tmp < d_min:
                    d_min = d_tmp
                    u = f_task
                elif d_tmp == d_min:
                    d_min = d_tmp
                    #print(u)
                    u = which_better(u, f_task, gf, load[k], capacity, depot)
                    #print(u)
            if d_min == float('inf'):
                break
            #if u != None:
            R[k].append(u)
            free_task.remove(u)
            free_task.remove((u[1],u[0]))
            cost[k] += graph[u[0]][u[1]]['cost'] + d_min
            load[k] += graph[u[0]][u[1]]['demand']
            end = u[1]
        cost[k] += graph.get_shortest_path(u[1], depot)[1]
        #print("############ end #############")
    return R, load, cost

### Augment-merge

In [12]:
def concate_circles(graph, circle1, circle2, idx1, idx2):
    """
    params: circle: the path
    return [circle_list, overpapping_best, cost_largest]
    """
    overlapping_best = (0, 0)
    cost_largest = 0 # largest saving cost
    for p1, val1 in enumerate(circle1[-1:-1-idx1:-1]):
        for p2, val2 in enumerate(circle2[0:idx2]):
            if val1 == val2:
                cost_save_1 = Graph.calculate_path_cost(gf,circle1[-1:-1-p1-1])
                cost_save_2 = Graph.calculate_path_cost(gf,circle2[0:p2+1])
                save_total = cost_save_1 + cost_save_2
                if cost_largest<save_total:
                    overlapping_best = (p1,p2)
                    cost_largest = save_total
#     if cost_largest is 0:
#         print("gg", circle1, circle2, idx1, idx2)
    return circle1[:-1-overlapping_best[0]]+circle2[overlapping_best[1]:], overlapping_best, cost_largest

def get_first_last_req(circle):
    """
    params circle: dict
    """
    first, last = 0, 0
    for idx, val in enumerate(circle['circle'][-1::-1]):
        required = False
        for required_e in circle['aq_set']:
            if val in required_e:
                last = idx
                # print(val, required_e)
                required = True
                break
        if required:
            break
    for idx, val in enumerate(circle['circle']):
        required = False
        for required_e in circle['aq_set']:
            if val in required_e:
                first = idx
                required = True
                break
        if required:
            break
    return first,last

def merge_circles(graph, circle1, circle2):
    if circle1['load'] + circle2['load'] > capacity:
        return None
    idx1 = get_first_last_req(circle1)
    idx2 = get_first_last_req(circle2)
    # print(idx1,idx2)
    results = list()
    results.append(concate_circles(graph, circle1['circle'], circle2['circle'], idx1[1], idx2[0]))
    results.append(concate_circles(graph, circle1['circle'][::-1], circle2['circle'], idx1[0], idx2[0]))
    results.append(concate_circles(graph, circle2['circle'], circle1['circle'], idx2[1], idx1[0]))
    results.append(concate_circles(graph, circle2['circle'], circle1['circle'][::-1], idx2[1], idx1[1]))
    max_save = 0
    new_circle = None
    for result in results:
        if result[2]>=max_save:
            new_circle = result[0]
            max_save = result[2]
    if new_circle == None:
        # print(results)
        return None
    return {'circle':new_circle,'cost':circle1['cost']+circle2['cost']-max_save, 'saving': max_save, 
            'load':circle1['load'] + circle2['load'], 'aq_set':circle1['aq_set'].union(circle2['aq_set'])}

In [13]:

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return izip(a, b)

def is_inclueded(path, target):
    inv_target =  (target[1], target[0])
    for edge in pairwise(path):
        # print(target, edge)
        if target == edge or inv_target == edge:
            return True
    return False

def augment_merge(graph, depot):
    # init routes
    k = 0
    free_tasks = set(graph.get_tasks_unique())
    R = defaultdict(dict)
    # cost = defaultdict(dict)
    for f_task in free_tasks:
        x = graph.get_shortest_path(depot, f_task[0])
        y = graph.get_shortest_path(f_task[1], depot)
        if x[1]==0:
            x = ([depot,],y[1])
        if y[1]==0:
            y[0] = ([depot,],x[1])
        R[k]['circle'] = x[0]+y[0]
        R[k]['cost'] = x[1]+y[1]
        R[k]['load'] = graph[f_task[0]][f_task[1]]['demand']
        R[k]['aq_set'] = set()
        R[k]['aq_set'].add(f_task)
        #print(f_task, R[k], cost[k])
        k += 1
    # augment phase
    R_sorted = sorted(R.iteritems(), key=lambda (k,v):len(v['circle']), reverse=True)
    deleted = set()
    for idx, circle in enumerate(R_sorted):
        for smaller_circle in R_sorted[idx+1:]:
            if smaller_circle[0] in deleted:
                continue
            if circle[1]['load'] + smaller_circle[1]['load'] > capacity:
                continue
            #print(smaller_circle[1][1]['aq_set'])
            flag_include = True
            for edge in smaller_circle[1]['aq_set']:
                #print(circle)
                if not is_inclueded(circle[1]['circle'],edge):
                    flag_include = False
            if flag_include:
                deleted.add(smaller_circle[0])
                circle[1]['aq_set'] = circle[1]['aq_set'].union(smaller_circle[1]['aq_set'])
                circle[1]['load'] += smaller_circle[1]['load']
                #print("[oK]", circle, smaller_circle)
    R_aug = list()
    for circle in R_sorted:
        if circle[0] not in deleted:
            circle[1]['delete'] = False
            R_aug.append(circle[1])
    '''
    no augment phase
    
    deleted = set()
    R_sorted = sorted(R.iteritems(), key=lambda (k,v):len(v['circle']), reverse=True)
#     for idx, circle in enumerate(R_sorted):
#         for edge in free_tasks:
#             if is_inclueded(circle[1]['circle'],edge):
#                 circle[1]['aq_set'].add(edge)
#                 circle[1]['load'] += graph[edge[0]][edge[1]]['demand']
#                 if circle[1]['load'] > capacity:
#                     print(circle[1]['load'])
#                     deleted.add(circle[0])
#                     break
    #print(R_sorted)
    R_aug = list()
    for circle in R_sorted:
        circle[1]['delete'] = False
        R_aug.append(circle[1])
'''
    # merge phase
    while True:
        merge_res = list()
        merge_next = list()
        for idx1, circle1 in enumerate(R_aug):
            for idx2, circle2 in enumerate(R_aug[idx1+1:]):
                #print(graph, circle1, circle2)
                tmp = merge_circles(graph, circle1, circle2)
                if tmp:
                    merge_res.append((tmp, idx1, idx1+idx2+1))
                    #print(tmp, idx1, idx2+idx1+1, circle1, circle2)
        if len(merge_res) is 0:
            break
        merge_res = sorted(merge_res, key=lambda (tmp,idx1,idx2):tmp['saving'], reverse=True)
        for merger in merge_res:
            if not R_aug[merger[1]]['delete'] and not R_aug[merger[2]]['delete']:
                # print(merger[1],merger[2])
                R_aug[merger[1]]['delete'] = True
                R_aug[merger[2]]['delete'] = True
                del(merger[0]['saving'])
                merger[0]['delete'] = False
                merge_next.append(merger[0])
        for circle in R_aug:
            if not circle['delete']:
                # print(circle)
                merge_next.append(circle)
        R_aug = merge_next
    R_final = R_aug
    """
    augment
    """
    """
    for idx, circle in enumerate(R_aug):
        for edge in free_tasks:
            if is_inclueded(circle['circle'],edge):
                if circle['load'] + graph[edge[0]][edge[1]]['demand'] > capacity:
                    continue
                circle['aq_set'].add(edge)
                circle['load'] += graph[edge[0]][edge[1]]['demand']
    deleted = set()
    R_final = sorted(R_final, key=lambda v:len(v['circle']), reverse=True)
    for idx, circle in enumerate(R_final):
        for id2, smaller_circle in enumerate(R_final[idx+1:]):
            if id2 + idx + 1 in deleted:
                continue
            flag_include = True
            for edge in smaller_circle['aq_set']:
                #print(circle)
                if edge not in circle['aq_set']:
                    flag_include = False
            if flag_include:
                deleted.add(id2 + idx + 1)
                circle['aq_set'] = circle['aq_set'].union(smaller_circle['aq_set'])
                circle['load'] += smaller_circle['load']
                #print("[oK]", circle, smaller_circle)
    result = list()
    for idx, circle in enumerate(R_final):
        if idx not in deleted:
            result.append(circle)
    """
    return R_final

def am_to_chromesome(am_solution):
    result = list()
    for item in am_solution:
        circle = item['circle']
        for p in pairwise(circle):
            tmp = Graph.get_unique_edge(p)
            if tmp in item['aq_set'] and p not in result and p[::-1] not in result:
                result.append(p)
    return result

In [14]:
def random_init(graph, depot):
    tasks = graph.get_tasks_unique()
    solution = list(tasks)
    shuffle(solution)
    def flip(task):
        if random.random() < 0.5:
            return (task[1],task[0])
        else:
            return task
    solution = map(flip, solution)
    return solution

In [15]:
init_solution1 = path_scanning(gf, depot)
init_solution2 = augment_merge(gf, depot)
def get_cost(solution):
    cost = 0
    demand = 0
    for circle in solution:
        cost += circle['cost']
        demand += circle['load']
    print(cost, demand)
get_cost(init_solution2)

print(init_solution1)
# print(init_solution2)

(10668, 1394)
(defaultdict(<type 'dict'>, {1: [(1, 116), (116, 117), (117, 119), (117, 2), (118, 114), (114, 113), (113, 112), (112, 110), (110, 107), (107, 108), (107, 112)], 2: [(110, 111), (107, 106), (106, 105), (105, 104), (104, 102), (66, 67), (67, 68), (67, 69), (69, 71), (71, 72), (72, 73), (73, 44), (44, 45), (45, 34)], 3: [(87, 86), (86, 85), (85, 84), (84, 82), (82, 80), (80, 79), (79, 78), (78, 77), (77, 46), (46, 43), (43, 37), (37, 36), (36, 38), (38, 39), (39, 40)], 4: [(108, 109), (66, 62), (62, 63), (63, 64), (64, 65), (56, 55), (55, 54), (55, 140), (49, 48)], 5: [(124, 126), (126, 130), (43, 44), (34, 139), (139, 33), (33, 11), (11, 12), (12, 13), (20, 22)], 6: [(95, 96), (96, 97), (97, 98), (140, 49), (11, 8), (8, 9), (8, 6), (6, 5), (13, 14), (24, 25), (27, 28)], 7: [(11, 27), (27, 25), (24, 20), (28, 29), (28, 30), (30, 32)]}), defaultdict(<type 'dict'>, {1: 210, 2: 210, 3: 208, 4: 209, 5: 210, 6: 207, 7: 140}), defaultdict(<type 'dict'>, {1: 404, 2: 658, 3: 642, 4

In [16]:
solution_verify(chromesome_partition(ps_to_chromesome(path_scanning(gf, depot)[0]), gf), gf)[2]

NameError: name 'solution_verify' is not defined

In [17]:
p1 = ps_to_chromesome(init_solution1[0])
p2 = am_to_chromesome(init_solution2)
print(p1, len(p1))
print(p2, len(p2))
# print(init_solution2)

([(1, 116), (116, 117), (117, 119), (117, 2), (118, 114), (114, 113), (113, 112), (112, 110), (110, 107), (107, 108), (107, 112), (110, 111), (107, 106), (106, 105), (105, 104), (104, 102), (66, 67), (67, 68), (67, 69), (69, 71), (71, 72), (72, 73), (73, 44), (44, 45), (45, 34), (87, 86), (86, 85), (85, 84), (84, 82), (82, 80), (80, 79), (79, 78), (78, 77), (77, 46), (46, 43), (43, 37), (37, 36), (36, 38), (38, 39), (39, 40), (108, 109), (66, 62), (62, 63), (63, 64), (64, 65), (56, 55), (55, 54), (55, 140), (49, 48), (124, 126), (126, 130), (43, 44), (34, 139), (139, 33), (33, 11), (11, 12), (12, 13), (20, 22), (95, 96), (96, 97), (97, 98), (140, 49), (11, 8), (8, 9), (8, 6), (6, 5), (13, 14), (24, 25), (27, 28), (11, 27), (27, 25), (24, 20), (28, 29), (28, 30), (30, 32)], 75)
([(20, 22), (13, 14), (28, 30), (30, 32), (27, 11), (66, 62), (62, 63), (64, 63), (55, 140), (140, 49), (69, 67), (67, 68), (66, 67), (49, 48), (69, 71), (71, 72), (72, 73), (73, 44), (8, 6), (6, 5), (107, 110), 

## Metaheuristics
### optimal chromesome partition

In [18]:
def chromesome_partition(chromesome, graph):
    '''
    transform the solution from chromesome format to partitioned solution optimally
    input: chromesome and graph
    output: 
    '''
    # generate Auxiliary graph
    aux_graph = DiGraph()
    start_cost = list()
    end_cost = list()
    arc_cost = dict()
    aq_edges = list()
    edge_demand = list()
    pre_task = None
    for idx, task in enumerate(chromesome):
        start_cost.append(graph.get_shortest_path(depot, task[0])[1])
        aq_edges.append((graph[task[0]][task[1]]['cost'], graph[task[0]][task[1]]['demand']))
        end_cost.append(graph.get_shortest_path(task[1], depot)[1])
        if pre_task is not None:
            arc_cost[(idx-1, idx)] = graph.get_shortest_path(pre_task[1], task[0])[1]
        pre_task = task

    # print(start_cost, end_cost, arc_cost, edge_cost)
    for node1 in range(len(chromesome)):
        cost = start_cost[node1]
        load = 0
        for node2 in range(node1 + 1,len(chromesome) + 1):
            if load + aq_edges[node2-1][1] > capacity:
                break
            load += aq_edges[node2-1][1]
            cost += aq_edges[node2-1][0]
            aux_graph.add_weighted_edge((node1, node2), cost + end_cost[node2-1])
            if node2!= len(chromesome):
                cost += arc_cost[(node2-1, node2)]
            #print(node1, node2)
            #aux_graph.add_weighted_edge()

    result = aux_graph.get_shortest_path((0,len(chromesome)))
    partitioned = list()
    for pair in pairwise(result[0]):
        partitioned.append(chromesome[pair[0]:pair[1]])
    return partitioned

In [19]:
partitioned1 = chromesome_partition(p1, gf)
partitioned2 = chromesome_partition(p2, gf)

### Order crossover

In [20]:
def cxOrdered(x1, x2):
    """Executes an ordered crossover (OX) on the input
    individuals. The two individuals are modified in place. This crossover
    expects :term:`sequence` individuals of indices, the result for any other
    type of individuals is unpredictable.
    
    :param x1: The first individual participating in the crossover.
    :param x2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    Moreover, this crossover generates holes in the input
    individuals. A hole is created when an attribute of an individual is
    between the two crossover points of the other individual. Then it rotates
    the element so that all holes are between the crossover points and fills
    them with the removed elements in order. For more details see
    [Goldberg1989]_.
    
    This function uses the :func:`~random.sample` function from the python base
    :mod:`random` module.
    
    .. [Goldberg1989] Goldberg. Genetic algorithms in search, 
       optimization and machine learning. Addison Wesley, 1989
    """
    ind1, ind2 = copy.copy(x1), copy.copy(x2)
    size = min(len(ind1), len(ind2))
    a, b = random.sample(xrange(size), 2)
    # print(a,b)
    if a > b:
        a, b = b, a

    holes1, holes2 = set(), set()
    for i in range(size):
        if i < a or i > b:
            holes1.add(Graph.get_unique_edge(ind2[i]))
            holes2.add(Graph.get_unique_edge(ind1[i]))
#             holes1[ind2[i]] = False
#             holes2[ind1[i]] = False
    
    # We must keep the original values somewhere before scrambling everything
    temp1, temp2 = ind1, ind2
    k1 , k2 = b + 1, b + 1
    for i in range(size):
        if Graph.get_unique_edge(temp1[(i + b + 1) % size]) in holes1:
            ind1[k1 % size] = temp1[(i + b + 1) % size]
            k1 += 1
        
        if Graph.get_unique_edge(temp2[(i + b + 1) % size]) in holes2:
            ind2[k2 % size] = temp2[(i + b + 1) % size]
            k2 += 1
    
    # Swap the content between a and b (included)
    for i in range(a, b + 1):
        ind1[i], ind2[i] = ind2[i], ind1[i]
    
    return ind1, ind2



In [22]:
def cx_LCSX(x1, x2):
    pass

In [23]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
def get_lcs(a, b):
    lengths = [[0 for j in range(len(b)+1)] for i in range(len(a)+1)]
    pp.pprint(lengths)
    # row 0 and column 0 are initialized to 0 already
    for i, x in enumerate(a):
        for j, y in enumerate(b):
            if x == y:
                lengths[i+1][j+1] = lengths[i][j] + 1
            else:
                lengths[i+1][j+1] = max(lengths[i+1][j], lengths[i][j+1])
    pp.pprint(lengths)
    result_set = []
    longest = []
    get_longest_from_matrix(lengths, longest, len(a), len(b), a,b,result_set)
    return result_set

def get_longest_from_matrix(matrix, longest, x, y, a, b, result_set):
    '''
    time complexity too high
    need rewrite
    https://www.cnblogs.com/wb-DarkHorse/archive/2012/11/15/2772520.html
    '''
    # read the substring out from the matrix
    while x!=0 and y!=0:
        print(x,y)
        if a[x-1] == b[y-1]:
            print(1)
            longest.append(a[x-1])
            x -= 1
            y -= 1
        else:
            if matrix[x-1][y] > matrix[x][y-1]:
                print(2)
                x -= 1
            elif  matrix[x-1][y] < matrix[x][y-1]:
                print(3)
                y -= 1
            else:
                print(4)
                get_longest_from_matrix(matrix, longest, x-1, y, a, b, result_set)
                get_longest_from_matrix(matrix, longest, x, y-1, a, b, result_set)
                return
    result_set.append(longest)

In [24]:
def mutation(idv):
    a, b = random.sample(xrange(len(idv['chromesome'])), 2)
    print('mutation',a,b)
    tmp = idv['chromesome'][a]
    idv['chromesome'][a] = idv['chromesome'][b]
    idv['chromesome'][b] = tmp
    idv['partition'] = chromesome_partition(idv['chromesome'], gf)
    idv['fitness'] = solution_verify(idv['partition'],gf)[2]
    return idv

def main_process():
    ps_random = 18
    ps = 20
    # seq = list(range(20))
    P = defaultdict(dict)
    MAX_ITERATION = 5000
    MAX_RESTART = 10
    n_iteration = 0
    x_bsf = (-1, float('inf'))
    P[0]['chromesome'] = p1
    P[0]['partition'] = chromesome_partition(p1, gf)
    fitness = solution_verify(P[0]['partition'],gf)[2]
    P[0]['fitness'] = fitness
    if x_bsf[1] > fitness:
        x_bsf = (0,fitness)
        print(x_bsf)
    P[1]['chromesome'] = p2
    P[1]['partition'] = chromesome_partition(p1, gf)
    fitness = solution_verify(P[1]['partition'],gf)[2]
    P[1]['fitness'] = fitness
    if x_bsf[1] > fitness:
        x_bsf = (1,fitness)
        print(x_bsf)
    for x in range(2, 2 + ps_random):
        p = random_init(gf,depot)
        P[x]['chromesome'] = p
        P[x]['partition'] = chromesome_partition(p, gf)
        P[x]['fitness'] = solution_verify(P[x]['partition'],gf)[2]
        #print(p)
        if x_bsf[1] > P[x]['fitness']:
            x_bsf = (x,P[x]['fitness'])
            print(x_bsf)
    return P
    while n_iteration < MAX_ITERATION:
        a, b = random.sample(xrange(ps), 2)
        if P[a]['fitness'] < P[b]['fitness']:
            tmp, b = b, a
            a = tmp
        child = cxOrdered(P[a]['chromesome'], P[b]['chromesome'])
        child = random.choice(child)
        new_partition = chromesome_partition(child, gf)
        new_fitness =  solution_verify(new_partition,gf)[2]
        #print(new_fitness)
        if new_fitness < P[a]['fitness']:
            P[a]['chromesome'] = child
            P[a]['partition'] = new_partition
            P[a]['fitness'] = new_fitness
            print(new_fitness)
            if new_fitness < x_bsf[1]:
                print(x_bsf)
                x_bsf = (a, new_fitness)
            if child ==  P[b]['chromesome']:
                mutation(P[a])
                
        n_iteration += 1
    #print(P)

### local search

In [328]:
def soft_evaluate(idv):
    '''
    ::params: population individual
    ::return: (soft_feasibility, feasibility, cost)
    '''
    pass

def copy_partition(route):
    '''
    pesudo-deepcopy for the route
    '''
    new_route = list()
    for trip in route:
        new_trip = list()
        for task in trip:
            new_trip.append(task)
        new_route.append(new_trip)
    return new_route

def method_inverse(idv):
    route = idv['partition']
    for idx1,trip in enumerate(route):
        for idx2,task in enumerate(trip):
            tmp = copy_partition(route)
            # print(idx1, idx2, tmp)
            tmp[idx1][idx2] = (task[1], task[0])
            newfitness = solution_verify(tmp, gf)[2]
            # print(newfitness)
            if newfitness < idv['fitness']:
                return (True, tmp, newfitness)

def method_move(idv):
    '''
    move only one task
    '''
    route = idv['partition']
    count = 0
    total_time = 0
    for idx1,trip in enumerate(route):
        last_task = len(trip) - 1
        for idx2,task in enumerate(trip):
            if idx2 == 0:
                prev_p = depot
            else:
                prev_p = trip[idx2-1][1]
            if last_task == idx2:
                next_p = depot
            else:
                next_p = trip[idx2+1][0]
            cost01 = gf.get_shortest_path(prev_p, trip[idx2][0])[1]
            cost02 = gf.get_shortest_path(trip[idx2][1], next_p)[1]
            cost03 = gf.get_shortest_path(prev_p, next_p)[1]
            save_cost1 =  cost01 + cost02 - cost03
            tmp1 = copy_partition(route)
            del tmp1[idx1][idx2]
            for idx3,trip2 in enumerate(tmp1):
                last_task2 = len(trip2)
                for idx4 in range(len(trip2)+1):
                    # print(trip2, idx4)
                    if idx4 == 0:
                        prev_p = depot
                    else:
                        prev_p = trip2[idx4-1][1]
                    if last_task2 == idx4:
                        next_p = depot
                    else:
                        next_p = trip2[idx4][0]
                    # print(prev_p, task)
                    cost1 = gf.get_shortest_path(prev_p, task[0])[1]
                    cost2 = gf.get_shortest_path(task[1], next_p)[1]
                    cost3 = gf.get_shortest_path(prev_p, next_p)[1]
                    save_cost2 =  save_cost1 - (cost1 + cost2 - cost3)
                    if save_cost2 > 0:
                        tmp2 = copy_partition(tmp1)
                        tmp2[idx3].insert(idx4,task)
                        newfitness = idv['fitness'] - save_cost2
                        #print("good", cost01, cost02, cost03 ,save_cost1, save_cost2, cost1,cost2,cost3, prev_p, next_p)
                        count += 1
                        # return (True, tmp2, newfitness)
                    # inverse insert
                    cost1 = gf.get_shortest_path(prev_p, task[1])[1]
                    cost2 = gf.get_shortest_path(task[0], next_p)[1]
                    cost3 = gf.get_shortest_path(prev_p, next_p)[1]
                    save_cost2 =  save_cost1 - (cost1 + cost2 - cost3)
                    if save_cost2 > 0:
                        tmp2 = copy_partition(tmp1)
                        tmp2[idx3].insert(idx4,task[::-1])
                        newfitness = idv['fitness'] - save_cost2
                        count += 1
    return count
                    

def local_search(idv):
    '''
    ::params: population individual
    ::output: new individual
    '''
    print(idv)
    #ethod_inverse(idv)
    # method_swap(idv)
    print(method_two_opt(idv))

In [280]:
def method_move_two(idv):
    '''
    move only one task
    '''
    route = idv['partition']
    count = 0
    total_time = 0
    for idx1,trip in enumerate(route):
        last_task = len(trip) - 2
        for idx2,pair in enumerate(pairwise(trip)):
            if idx2 == 0:
                prev_p = depot
            else:
                prev_p = trip[idx2-1][1]
            if last_task == idx2:
                next_p = depot
            else:
                next_p = trip[idx2+2][0]
            cost01 = gf.get_shortest_path(prev_p, trip[idx2][0])[1]
            cost02 = gf.get_shortest_path(trip[idx2+1][1], next_p)[1]
            cost03 = gf.get_shortest_path(prev_p, next_p)[1]
            save_cost1 =  cost01 + cost02 - cost03
            tmp1 = copy_partition(route)
            del tmp1[idx1][idx2]
            del tmp1[idx1][idx2]
            for idx3,trip2 in enumerate(tmp1):
                last_task2 = len(trip2)
                for idx4 in range(len(trip2)+1):
                    # print(trip2, idx4)
                    if idx4 == 0:
                        prev_p = depot
                    else:
                        prev_p = trip2[idx4-1][1]
                    if last_task2 == idx4:
                        next_p = depot
                    else:
                        next_p = trip2[idx4][0]
                    # print(prev_p, task)
                    cost1 = gf.get_shortest_path(prev_p, pair[0][0])[1]
                    cost2 = gf.get_shortest_path(pair[1][1], next_p)[1]
                    cost3 = gf.get_shortest_path(prev_p, next_p)[1]
                    save_cost2 =  save_cost1 - (cost1 + cost2 - cost3)
                    
#                     if count % 100 == 0:
#                         print(count)
                    if save_cost2 > 0:
                        tmp2 = copy_partition(tmp1)
                        print(list(pair))
                        tmp2[idx3][idx4:idx4] = list(pair)
                        newfitness = idv['fitness'] - save_cost2
                        print("good", cost01, cost02, cost03 ,save_cost1, save_cost2, cost1,cost2,cost3, prev_p, next_p)
                        count += 1
                        return (True, tmp2, newfitness)
    return count

In [322]:
def method_swap(idv):
    '''
    swap two tasks
    need to add inverse insert
    '''
    route = idv['partition']
    count = 0
    total_time = 0
    for idx1,trip in enumerate(route):
        last_task = len(trip) - 1
        for idx2,task1 in enumerate(trip):
            if idx2 == 0:
                prev_p1 = depot
            else:
                prev_p1 = trip[idx2-1][1]
            if last_task == idx2:
                next_p1 = depot
            else:
                next_p1 = trip[idx2+1][0]
            cost01 = gf.get_shortest_path(prev_p1, task1[0])[1]
            cost02 = gf.get_shortest_path(task1[1], next_p1)[1]
            scost1 =  cost01 + cost02
            tmp1 = copy_partition(route)
            del tmp1[idx1][idx2]
            # find another task
            idx3 = idx1
            if idx2 + 2 <= last_task:
                for offset,task2 in enumerate(trip[idx2+2:]):
                    # only for that trip, skip the consecutive task
                    idx4 = idx2 + offset + 2
                    prev_p2 = trip[idx4-1][1]
                    if last_task == idx4:
                        next_p2 = depot
                    else:
                        next_p2 = trip[idx4+1][0]
                    # print(prev_p, task)
                    cost1 = gf.get_shortest_path(prev_p2, task2[0])[1]
                    cost2 = gf.get_shortest_path(task2[1], next_p2)[1]
                    scost2 =  cost1 + cost2
                    cost11 = gf.get_shortest_path(prev_p1, task2[0])[1]
                    cost12 = gf.get_shortest_path(task2[1], next_p1)[1]
                    cost21 = gf.get_shortest_path(prev_p2, task1[0])[1]
                    cost22 = gf.get_shortest_path(task1[1], next_p2)[1]
                    final_save = scost1 + scost2 - cost11 - cost12 - cost21 - cost22
                    if final_save > 0:
                        tmp2 = copy_partition(tmp1)
                        tmp2[idx1].insert(idx2,task2)
                        del tmp2[idx3][idx4]
                        tmp2[idx3].insert(idx4,task1)
                        newfitness = idv['fitness'] - final_save
                        #print("good", cost01, cost02, cost03 ,save_cost1, save_cost2, cost1,cost2,cost3, prev_p, next_p)
                        count += 1
                        return (True, tmp2, newfitness)
            for offset,trip2 in enumerate(tmp1[idx1+1:]):
                idx3 = offset + idx1 + 1
                last_task2 = len(trip2) - 1
                for idx4, task2 in enumerate(trip2):
                    # print(trip2, idx4)
                    if idx4 == 0:
                        prev_p2 = depot
                    else:
                        prev_p2 = trip2[idx4-1][1]
                    if last_task2 == idx4:
                        next_p2 = depot
                    else:
                        next_p2 = trip2[idx4+1][0]
                    # print(prev_p, task)
                    cost1 = gf.get_shortest_path(prev_p2, task2[0])[1]
                    cost2 = gf.get_shortest_path(task2[1], next_p2)[1]
                    scost2 =  cost1 + cost2
                    cost11 = gf.get_shortest_path(prev_p1, task2[0])[1]
                    cost12 = gf.get_shortest_path(task2[1], next_p1)[1]
                    cost21 = gf.get_shortest_path(prev_p2, task1[0])[1]
                    cost22 = gf.get_shortest_path(task1[1], next_p2)[1]
                    final_save = scost1 + scost2 - cost11 - cost12 - cost21 - cost22
#                     if count % 100 == 0:
#                         print(count)
                    if final_save > 0:
                        tmp2 = copy_partition(tmp1)
                        tmp2[idx1].insert(idx2,task2)
                        del tmp2[idx3][idx4]
                        tmp2[idx3].insert(idx4,task1)
                        newfitness = idv['fitness'] - final_save
                        #print("good",task1, task2, cost01, cost02, cost03 ,save_cost1, \
                        #      save_cost2, cost1,cost2,cost3, prev_p, next_p)
                        count += 1
                        #return (True, tmp2, newfitness)
    return count

In [367]:
def method_two_opt(idv):
    '''
    two opt method
    u-v, x-y
    => u-y, v-x
    => u-x, v-y
    '''
    route = idv['partition']
    count = 0
    total_time = 0
    for idx1,trip in enumerate(route):
        last_task = len(trip) - 2
        for idx2,pair1 in enumerate(pairwise(trip)):
            if idx2 == 0:
                prev_p1 = depot
            else:
                prev_p1 = trip[idx2-1][1]
            if last_task == idx2:
                next_p1 = depot
            else:
                next_p1 = trip[idx2+2][0]
            cost01 = gf.get_shortest_path(prev_p1, pair1[0][0])[1]
            cost02 = gf.get_shortest_path(pair1[1][1], next_p1)[1]
            cost03 = gf.get_shortest_path(pair1[0][1], pair1[1][0])[1]
            original_cost0 = cost01 + cost02 + cost03
            tmp1 = copy_partition(route)
            del tmp1[idx1][idx2]
            del tmp1[idx1][idx2]
            if idx2 < last_task - 5:
                for offset, pair in enumerate(pairwise(trip[idx2+3:])):
                    pass
            for offset,trip2 in enumerate(tmp1[idx1+1:]):
                idx3 = offset + idx1 + 1
                last_task2 = len(trip2) - 2
                for idx4, pair2 in enumerate(pairwise(trip2)):
                    if idx4 == 0:
                        prev_p2 = depot
                    else:
                        prev_p2 = trip2[idx4-1][1]
                    if last_task2 == idx4:
                        next_p2 = depot
                    else:
                        next_p2 = trip2[idx4+2][0]
                    # print(prev_p, task)
                    cost11 = gf.get_shortest_path(prev_p2, pair2[0][0])[1]
                    cost12 = gf.get_shortest_path(pair2[1][1], next_p2)[1]
                    cost13 = gf.get_shortest_path(pair2[0][1], pair2[1][0])[1]
                    original_cost1 = original_cost0 + cost11 + cost12 + cost13
                    
                    # u-v, x-y  => u-y, v-x
                    cost21 = cost01 # -u
                    cost22 = gf.get_shortest_path(pair2[1][1], next_p1)[1] # y-
                    cost23 = gf.get_shortest_path(pair1[0][1], pair2[1][0])[1] # u-y
                    cost31 = gf.get_shortest_path(prev_p2, pair1[1][0])[1] # -v
                    cost32 = gf.get_shortest_path(pair2[0][1], next_p2)[1] # x-
                    cost33 = gf.get_shortest_path(pair1[1][1], pair2[0][0])[1]
                    newcost = cost21 + cost22 + cost23 + cost31 + cost32 + cost33
                    if original_cost1- newcost > 0:
                        tmp2 = copy_partition(tmp1)
                        tmp2[idx1][idx2:idx2] = list((pair1[0],pair2[1]))
                        del tmp2[idx3][idx4]
                        del tmp2[idx3][idx4]
                        tmp2[idx3][idx4:idx4] = list((pair1[1],pair2[0]))
                        newfitness = idv['fitness'] - (original_cost1- newcost)
                        print("good", original_cost0, original_cost1,newcost, prev_p1, next_p1, prev_p2, next_p2)
                        count += 1
                        return (True, tmp2, newfitness)
                    # u-v, x-y  => u-x, v-y
                    cost21 = cost01 # -u
                    cost22 = gf.get_shortest_path(pair2[0][1], next_p1)[1] # x-
                    cost23 = gf.get_shortest_path(pair1[0][1], pair2[0][0])[1] # u-x
                    cost31 = gf.get_shortest_path(prev_p2, pair1[1][0])[1] # -v
                    cost32 = gf.get_shortest_path(pair2[1][1], next_p2)[1] # y-
                    cost33 = gf.get_shortest_path(pair1[1][1], pair2[1][0])[1] # v-y
                    newcost = cost21 + cost22 + cost23 + cost31 + cost32 + cost33
                    if original_cost1- newcost > 0:
                        tmp2 = copy_partition(tmp1)
                        tmp2[idx1][idx2:idx2] = list((pair1[0],pair2[0]))
                        del tmp2[idx3][idx4]
                        del tmp2[idx3][idx4]
                        tmp2[idx3][idx4:idx4] = list((pair1[1],pair2[1]))
                        newfitness = idv['fitness'] - (original_cost1- newcost)
                        print("good", original_cost0, original_cost1,newcost, prev_p1, next_p1, prev_p2, next_p2)
                        count += 1
                        return (True, tmp2, newfitness)
    return count

In [368]:
test_p = dict()
test_p['partition'] = chromesome_partition(p1, gf)
test_p['fitness'] = solution_verify(test_p['partition'], gf)[2]
start = time.time()
local_search(test_p)
print(time.time() - start)

{'partition': [[(1, 116), (116, 117), (117, 119), (117, 2), (118, 114), (114, 113), (113, 112), (112, 110), (110, 107), (107, 108), (107, 112)], [(110, 111), (107, 106), (106, 105), (105, 104), (104, 102), (66, 67), (67, 68), (67, 69), (69, 71), (71, 72), (72, 73), (73, 44), (44, 45), (45, 34)], [(87, 86), (86, 85), (85, 84), (84, 82), (82, 80), (80, 79), (79, 78), (78, 77), (77, 46), (46, 43), (43, 37), (37, 36), (36, 38), (38, 39), (39, 40)], [(108, 109), (66, 62), (62, 63), (63, 64), (64, 65), (56, 55), (55, 54), (55, 140), (49, 48)], [(124, 126), (126, 130), (43, 44), (34, 139), (139, 33), (33, 11), (11, 12), (12, 13), (20, 22)], [(95, 96), (96, 97), (97, 98), (140, 49), (11, 8), (8, 9), (8, 6), (6, 5), (13, 14), (24, 25), (27, 28)], [(11, 27), (27, 25), (24, 20), (28, 29), (28, 30), (30, 32)]], 'fitness': 6152}
('good', 105, 451, 442, 107, 1, 1, 62)
(True, [[(1, 116), (116, 117), (117, 119), (117, 2), (118, 114), (114, 113), (113, 112), (112, 110), (110, 107), (107, 108), (108, 10

In [345]:
solution_verify(test_p['partition'],gf)

([210, 210, 208, 209, 210, 207, 140],
 [404, 1062, 1704, 2644, 3723, 5119, 6152],
 6152,
 1394)

In [363]:
gf.get_shortest_path(28, 1)[1]

405

In [27]:
test_p = chromesome_partition(p1, gf)
#test_p = main_process()[1]

In [369]:
x = [[(1, 116), (116, 117), (117, 119), (117, 2), (118, 114), (114, 113), (113, 112), (112, 110), (110, 107), (107, 108), (108, 109)], [(110, 111), (107, 106), (106, 105), (105, 104), (104, 102), (66, 67), (67, 68), (67, 69), (69, 71), (71, 72), (72, 73), (73, 44), (44, 45), (45, 34)], [(87, 86), (86, 85), (85, 84), (84, 82), (82, 80), (80, 79), (79, 78), (78, 77), (77, 46), (46, 43), (43, 37), (37, 36), (36, 38), (38, 39), (39, 40)], [(107, 112), (66, 62), (62, 63), (63, 64), (64, 65), (56, 55), (55, 54), (55, 140), (49, 48)], [(124, 126), (126, 130), (43, 44), (34, 139), (139, 33), (33, 11), (11, 12), (12, 13), (20, 22)], [(95, 96), (96, 97), (97, 98), (140, 49), (11, 8), (8, 9), (8, 6), (6, 5), (13, 14), (24, 25), (27, 28)], [(11, 27), (27, 25), (24, 20), (28, 29), (28, 30), (30, 32)]]
solution_verify(x,gf)

([300, 210, 208, 119, 210, 207, 140],
 [610, 1268, 1910, 2635, 3714, 5110, 6143],
 6143,
 1394)

## Solution-related

### output example

```
s 0,(1,2),(2,4),(4,1),0,0,(4,3),(3,1),0
q 15
```

In [299]:
def solution_verify(partitioned_solution, graph):
    loadlist = list()
    cost_total = 0
    costlist = list()
    for route in partitioned_solution:
        # costlist.append(route)
        load = 0
        prev_task = None
        cost_total += graph.get_shortest_path(route[0][0], depot)[1]
        # costlist.append(cost_total)
        for task in route:
            #print(task)
            if prev_task is not None:
                cost_total += graph.get_shortest_path(prev_task[1], task[0])[1]
                # costlist.append(cost_total)
            load += graph[task[0]][task[1]]['demand']
            cost_total += graph[task[0]][task[1]]['cost']
            # costlist.append(cost_total)
            prev_task = task
        loadlist.append(load)
        cost_total += graph.get_shortest_path(route[-1][1], depot)[1]
        costlist.append(cost_total)
    return loadlist, costlist,cost_total,sum(loadlist)

In [30]:
loadlist, costlist, cost_total, load_total = solution_verify(partitioned2, gf)
#print(load_total, cost_total)

In [337]:
start = time.time()
partitioned3 = chromesome_partition(random_init(gf,1), gf)
print(solution_verify(partitioned3, gf))
duration = time.time() -start
print(duration)

([132, 135, 187, 207, 183, 187, 110, 206, 47], [2242, 3767, 5469, 7204, 8914, 10983, 12509, 15824, 16267], 16267, 1394)
0.09042096138


In [21]:
def solution_to_string(partitioned_solution, graph):
    result = 's '
    for route in partitioned_solution:
        result+='0,'
        for task in route:
            result+='(%d,%d),' % task
        result+='0,'
    cost_total = solution_verify(partitioned_solution, graph)[2]
    result = result[:-1] + '\n'
    result += 'q %d' % cost_total
    return result

In [22]:
result_str = solution_to_string(partitioned2,gf)
print(result_str)

s 0,(20,22),(13,14),(28,30),(30,32),(27,11),(66,62),(62,63),(64,63),(55,140),(140,49),(69,67),0,0,(67,68),(66,67),(49,48),(69,71),(71,72),(72,73),(73,44),(8,6),(6,5),0,0,(107,110),(110,112),(110,111),(95,96),(96,97),(97,98),(107,108),(108,109),0,0,(124,126),(126,130),(43,37),(37,36),(36,38),(40,39),(39,38),0,0,(114,113),(113,112),(112,107),(107,106),(106,105),(105,104),(104,102),(64,65),(56,55),(55,54),(114,118),0,0,(1,116),(116,117),(117,2),(117,119),(87,86),(24,20),0,0,(34,139),(139,33),(33,11),(11,8),(8,9),(13,12),(12,11),0,0,(86,85),(85,84),(84,82),(82,80),(80,79),(79,78),(78,77),(77,46),(46,43),(43,44),(44,45),(45,34),(29,28),(28,27),(27,25),(25,24),0
q 7689


In [2]:
import re
class InputError(Exception):
    def __init__(self, message):
        self.msg = message

def string_extract(result_string):
    splitted = result_string.splitlines()
    solution = None
    solution = list()
    result_cost = 0
    try:
        for line in splitted:
            if len(line)<3:
                raise InputError('Length not right')
            if line[0] == 's':
                line = line[2:]
                pattern_route = r'(\([\d]+,[\d]+\),?)+'
                pattern_task = r'\(([\d]+),([\d]+)\)'
                for m in re.finditer(pattern_route, line):
                    route = list()
                    for m_sub in re.finditer(pattern_task, m.group(0)):
                        route.append((int(m_sub.group(1)), int(m_sub.group(2))))
                    solution.append(route)
            elif line[0] == 'q':
                quality = line.split(' ')
                try:
                    result_cost = int(quality[1])
                    # print(result_cost)
                except ValueError:
                    #Handle the exception
                    raise InputError('Please enter an integer')
    except InputError as err:
        print(err.msg)
    return solution, result_cost